In [ ]:
import newspaper
import requests
from bs4 import BeautifulSoup
import psycopg2

In [ ]:
blog = newspaper.Source('https://foxnomad.com/blog/')
blog.build()
blog.size()

In [ ]:
conn = psycopg2.connect(host='localhost', user='postgres', password='postgres', port=5432, database='blogscrapes')

In [ ]:
cur = conn.cursor()

In [ ]:
# This will get the links for all blogposts on foxnomad.com
pagelink = 'https://foxnomad.com/blog/'
while True:
    try:
        res = requests.get(pagelink)
        soup = BeautifulSoup(res.text)
        for post in soup.select('.post-title'):
            link = post.select('a')[0]['href']
            article = newspaper.Article(link, fetch_images=False)
            article.download()
            article.parse()
            article.nlp()
            thetext = article.text
            summary = article.summary
            sql = """
            INSERT INTO scrapes (url, text_data, summary) VALUES (%s, %s, %s);
            """
            cur.execute(sql, (link, thetext, summary))
            conn.commit()
        pagelink = soup.select('a.next')[0]['href']
    except IndexError:
        print('Seems like all has been scraped. Let\'s move on :\)')
        break